In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from google.colab import drive

os.chdir("drive/My Drive/IIITH/Advanced_NLP//Assignment_2")

In [3]:
eng_hi_Corpus = open("joint_Corpus.txt","r", encoding='utf-8')
eng_hi_Corpus = eng_hi_Corpus.read().splitlines()
df = pd.DataFrame(eng_hi_Corpus)
X_train, X_test = train_test_split(df[0:10000], test_size=0.1, random_state=0)

In [4]:
eng_hi_train = np.array(X_train)
eng_hi_test = np.array(X_test)

### create train file for English and Hindi by splitting the text by "!"
eng_Corpus_train = open("eng_Corpus_train.txt","w", encoding='utf-8')
hin_Corpus_train = open("hin_Corpus_train.txt","w", encoding='utf-8')

for i in range(eng_hi_train.shape[0]):
    eng_Corpus_train.write(eng_hi_train[i][0].split('|')[0] + '\n')
    hin_Corpus_train.write(eng_hi_train[i][0].split('|')[1] + '\n')
  
eng_Corpus_train.close()
hin_Corpus_train.close() 

### create test file for English and Hindi by splitting the text by "!"

eng_Corpus_test = open("eng_Corpus_test.txt","w", encoding='utf-8')
hin_Corpus_test = open("hin_Corpus_test.txt","w", encoding='utf-8')

for i in range(eng_hi_test.shape[0]):
    eng_Corpus_test.write(eng_hi_test[i][0].split('|')[0] + '\n')
    hin_Corpus_test.write(eng_hi_test[i][0].split('|')[1] + '\n')
  
eng_Corpus_test.close()
hin_Corpus_test.close() 

In [5]:
from keras.models import Model
from keras.models import load_model,save_model
from keras.layers import Input, LSTM, GRU, Dense, Embedding, Activation, dot, concatenate, Bidirectional, Attention
from tensorflow.keras import activations
from keras.preprocessing.text import Tokenizer
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping
import tensorflow_datasets as tfds
from keras.layers.wrappers import TimeDistributed, Bidirectional
from keras.backend import expand_dims,sum
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
import nltk

In [6]:
BATCH_SIZE = 32
LSTM_NODES =128
NUM_SENTENCES = 9000
MAX_NUM_WORDS = 10000
EMBEDDING_SIZE = 100

In [7]:
input_sentences = []
output_sentences = []
output_sentences_inputs = []

En_Corpus = open('eng_Corpus_train.txt','r',encoding = 'utf-8')
cnt = 0 
for line in En_Corpus.readlines():
    #print(line)
    if cnt < NUM_SENTENCES:
      input_sentences.append(line.strip('\n'))
    cnt = cnt+1
En_Corpus.close()

Hi_Corpus = open('hin_Corpus_train.txt','r',encoding = 'utf-8')
cnt = 0 
for line in Hi_Corpus.readlines():
  if cnt < NUM_SENTENCES:
    output_sentences.append(line.strip('\n') + ' <EOS>')
    output_sentences_inputs.append('<SOS> '+line.strip('\n') )
  cnt = cnt +1
Hi_Corpus.close()

print("num samples input:", len(input_sentences))
print("num samples output:", len(output_sentences))

num samples input: 9000
num samples output: 9000


In [8]:
input_tokenizer = Tokenizer(filters='')
input_tokenizer.fit_on_texts(input_sentences)
input_integer_seq = input_tokenizer.texts_to_sequences(input_sentences)

word2idx_inputs = input_tokenizer.word_index
print('Total unique words in the input: %s' % len(word2idx_inputs))

max_input_len = max(len(sen) for sen in input_integer_seq)
print("Length of longest sentence in input: %g" % max_input_len)
num_words_input = len(word2idx_inputs) + 1

Total unique words in the input: 8219
Length of longest sentence in input: 5


In [9]:
output_tokenizer = Tokenizer(filters='')
output_tokenizer.fit_on_texts(output_sentences + output_sentences_inputs)
output_integer_seq = output_tokenizer.texts_to_sequences(output_sentences)
output_input_integer_seq = output_tokenizer.texts_to_sequences(output_sentences)

word2idx_outputs = output_tokenizer.word_index
print('Total unique words in the output: %s' % len(word2idx_outputs))

num_words_output = len(word2idx_outputs) + 1
max_out_len = max(len(sen) for sen in output_integer_seq)
print("Length of longest sentence in the output: %g" % max_out_len)

Total unique words in the output: 8270
Length of longest sentence in the output: 29


In [10]:
input_text = input_sentences
target_text_input = output_sentences_inputs
target_text_output = output_sentences

input_integer_seq = input_tokenizer.texts_to_sequences(input_text)
encoder_input_sequences = pad_sequences(input_integer_seq, maxlen=max_input_len)
         
output_input_integer_seq = output_tokenizer.texts_to_sequences(target_text_input)
decoder_input_sequences = pad_sequences(output_input_integer_seq, maxlen=max_out_len, padding='post')
          
output_integer_seq = output_tokenizer.texts_to_sequences(target_text_output)
decoder_output_sequences = pad_sequences(output_integer_seq, maxlen=max_out_len, padding='post')
          
decoder_targets_one_hot = np.zeros((len(input_sentences),max_out_len,num_words_output),dtype='float32')

#print('decoder_targets_one_hot ',decoder_targets_one_hot.shape)


for i, d in enumerate(decoder_output_sequences):
  for t, word in enumerate(d):
    decoder_targets_one_hot[i, t, word] = 1
          

In [11]:
model = load_model('Seq2Seq_attentionB_assignment2_v1.h5')
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 5)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 5, 128)       1052160     input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 29)]         0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   [(None, 5, 256), (No 263168      embedding[0][0]                  
_______________________________________________________________________________________

In [12]:
#For Inference Purpose
encoder_inputs_placeholder = Input(shape=(max_input_len,))#model.get_layer('input_1')
encoder_embedding = model.get_layer('embedding')
encoder_bilstm = model.get_layer('bidirectional')

x = encoder_embedding(encoder_inputs_placeholder)
encoder_outputs,h_forward,c_forward, h_backward,c_backward = encoder_bilstm(x)
print('encoder_outputs shape:',encoder_outputs.shape)

h = concatenate([h_forward,h_backward],axis = 1)
c = concatenate([c_forward,c_backward],axis = 1)

encoder_states = [h, c]

#Define Encoder Model
encoder_model = Model(inputs = encoder_inputs_placeholder, outputs=[encoder_outputs, h, c])

encoder_outputs shape: (None, 5, 256)


In [13]:
decoder_lstm = model.get_layer('lstm_1')
decoder_state_input_h = Input(shape=(2*LSTM_NODES,),name='decoder_h_input')
decoder_state_input_c = Input(shape=(2*LSTM_NODES,),name='decoder_c_input')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

encoder_states_input = Input(shape=(max_input_len,LSTM_NODES*2), name = 'encoderstates_input')
print('encoder_states_input shape',encoder_states_input.shape)
encoder_outputs_inf = Input(shape=(max_input_len,LSTM_NODES*2))
print('encoder_outputs_inf shape',encoder_outputs_inf.shape)

print('decoder state input h shape:',decoder_state_input_h.shape)
decoder_inputs_single = Input(shape=(1,),name='decoder_single_input')
decoder_embedding = model.get_layer('embedding_1')

print(' decoder_inputs_single shape:',decoder_inputs_single.shape)
decoder_inputs_single_x = decoder_embedding(decoder_inputs_single)

decoder_outputs, decode_h, decode_c = decoder_lstm(decoder_inputs_single_x, initial_state=decoder_states_inputs)
decoder_states = [decode_h, decode_c ]
print('decoder_outputs shape:',decoder_outputs.shape)

encoder_states_input shape (None, 5, 256)
encoder_outputs_inf shape (None, 5, 256)
decoder state input h shape: (None, 256)
 decoder_inputs_single shape: (None, 1)
decoder_outputs shape: (None, 1, 256)


In [14]:
#Luong's attention
#attention = dot([decoder_outputs, encoder_states_input], axes=[2, 2])
#attention = Activation('softmax')(attention)
#context = dot([attention, encoder_states_input], axes=[2,1])
#decoder_combined_context = concatenate([context, decoder_outputs])

#Bhadnau's attention
w1_b = model.get_layer('dense')
w2_b = model.get_layer('dense_1')
v_b  = model.get_layer('dense_2')

encoder_h_timeaxis = expand_dims(encoder_states_input,1)
score = v_b(activations.tanh(w1_b(encoder_h_timeaxis) + (w2_b(encoder_outputs_inf)) ))
attention_softmax = activations.softmax(score,axis=1)
context = attention_softmax * encoder_outputs_inf
context_vector = sum(context,axis=1)
context_vector_timeaxis = expand_dims(context_vector,1)
decoder_combined_context = context_vector_timeaxis + decoder_outputs

decoder_dense = model.get_layer('dense_3')
decoder_outputs = decoder_dense(decoder_combined_context)

decoder_model = Model(
    [decoder_inputs_single] + decoder_states_inputs + [encoder_outputs_inf] + [encoder_states_input],
    [decoder_outputs] + decoder_states
)
decoder_model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoderstates_input (InputLayer [(None, 5, 256)]     0                                            
__________________________________________________________________________________________________
tf_op_layer_ExpandDims (TensorF [(None, 1, 5, 256)]  0           encoderstates_input[0][0]        
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 5, 256)]     0                                            
__________________________________________________________________________________________________
dense (Dense)                   multiple             2570        tf_op_layer_ExpandDims[0][0]     
_______________________________________________________________________________________

In [15]:
idx2word_input = {v:k for k, v in word2idx_inputs.items()}
idx2word_target = {v:k for k, v in word2idx_outputs.items()}

In [16]:
def translate_sentence(input_seq):
    encoder_outputs, h, c = encoder_model.predict(input_seq)
    
    states_value = [h,c]
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = word2idx_outputs['<sos>']
    eos = word2idx_outputs['<eos>']
    output_sentence = []

    for _ in range(max_out_len):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value + [encoder_outputs] + states_value)
        idx = np.argmax(output_tokens[0, 0, :])

        if eos == idx:
            break

        word = ''

        if idx > 0:
            word = idx2word_target[idx]
            output_sentence.append(word)

        target_seq[0, 0] = idx
        states_value = [h, c]

    return ' '.join(output_sentence)

In [17]:
# prediction on test data

input_sentences_test = []
output_sentences_test = []

En_Corpus_test = open('eng_Corpus_test.txt','r',encoding = 'utf-8')
cnt = 0 
for line in En_Corpus_test.readlines():
    #print(line)
    if cnt < NUM_SENTENCES:
      input_sentences_test.append(line.strip('\n'))
    cnt = cnt+1
En_Corpus_test.close()

Hi_Corpus_test = open('hin_Corpus_test.txt','r',encoding = 'utf-8')
cnt = 0 
for line in Hi_Corpus_test.readlines():
  if cnt < NUM_SENTENCES:
    output_sentences_test.append(line.strip('\n') + ' <EOS>')
  cnt = cnt +1
Hi_Corpus_test.close()


In [18]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
smooth = SmoothingFunction().method4
def calculate_bleu(text1,text2):
  hypothesis = text1.split()
  reference = text2.split()

  reference_list = [reference] # list of references for 1 sentence.
  bleu_score = nltk.translate.bleu_score.sentence_bleu(reference_list, hypothesis,smoothing_function = smooth)
  return bleu_score

In [19]:
test_pred_result = open('test_prediction','w',encoding = 'utf-8')
test_sent_bleu = {}
for s in range(len(input_sentences_test)):
  input_integer_seq = input_tokenizer.texts_to_sequences([input_sentences_test[s]])
  encoder_input_sequences = pad_sequences(input_integer_seq, maxlen=max_input_len)
  translation = translate_sentence(encoder_input_sequences)
  sent_bleu = calculate_bleu(output_sentences_test[s].strip('<EOS>'),translation)
  test_sent_bleu[s] = sent_bleu
  text = str(s) + ' ' + input_sentences_test[s] + '|' + output_sentences_test[s].strip('<EOS>') + 'Predicted: ' + translation + ' Bleu Score: ' + str(sent_bleu)
  test_pred_result.write(text + '\n')
test_pred_result.close()

  
test_cospus_score = np.sum(list(test_sent_bleu.values()))/len(input_sentences_test)
print('Test Corpus Bleu Score:',test_cospus_score)

Test Corpus Bleu Score: 0.16452619015883835


In [20]:
test_sent_bleu_sorted_index = sorted(test_sent_bleu, key=test_sent_bleu.get, reverse=True)

test_best = []
for i in test_sent_bleu_sorted_index[0:20]:
  test_best.append(i)

for s in test_best:
  input_integer_seq = input_tokenizer.texts_to_sequences([input_sentences_test[s]])
  encoder_input_sequences = pad_sequences(input_integer_seq, maxlen=max_input_len)
  translation = translate_sentence(encoder_input_sequences)
  sent_bleu = calculate_bleu(output_sentences_test[s].strip('<EOS>'),translation)
  test_sent_bleu[s] = sent_bleu
  text = str(s) + ' ' + input_sentences_test[s] + '|' + output_sentences_test[s].strip('<EOS>') + 'Predicted: ' + translation + ' Bleu Score: ' + str(sent_bleu)
  print(text)

28 - Who is he?|- वह कौन है? Predicted: - वह कौन है? Bleu Score: 1.0
133 "Sweetheart I swear on you."|"जानेमन मैं तुम पर कसम खाता हूँ।" Predicted: "जानेमन मैं तुम पर कसम खाता हूँ।" Bleu Score: 1.0
134 "Sweetheart I swear on you."|"जानेमन मैं तुम पर कसम खाता हूँ।" Predicted: "जानेमन मैं तुम पर कसम खाता हूँ।" Bleu Score: 1.0
253 man speaking native language:|पुरुष अपनी मातृभाषा में बोल रहा है: Predicted: पुरुष अपनी मातृभाषा में बोल रहा है: Bleu Score: 1.0
301 "Sweetheart I swear on you."|"जानेमन मैं तुम पर कसम खाता हूँ।" Predicted: "जानेमन मैं तुम पर कसम खाता हूँ।" Bleu Score: 1.0
326 What are you doing?|तुम क्या कर रहे हो? Predicted: तुम क्या कर रहे हो? Bleu Score: 1.0
507 "Sweetheart I swear on you."|"जानेमन मैं तुम पर कसम खाता हूँ।" Predicted: "जानेमन मैं तुम पर कसम खाता हूँ।" Bleu Score: 1.0
561 What are you doing?|तुम क्या कर रहे हो? Predicted: तुम क्या कर रहे हो? Bleu Score: 1.0
587 man speaking native language:|पुरुष अपनी मातृभाषा में बोल रहा है: Predicted: पुरुष अपनी मातृभाषा में